## Airline Industry Stocks during COVID-19 Pandemic First Wave

In [3]:
import pandas as pd # used for data frames
import numpy as np # used for math
import yfinance as yf # used for finance
import plotly.express as px # used for interactive graphing

In [5]:
# define the ticker symbol
my_ticker = 'DAL'

# get data on this ticker and put into ticker object
ticker = yf.Ticker(my_ticker)

# get the historical prices for this ticker
ticker_data = ticker.history(period='1d', start='2020-1-1', end='2020-10-1')

# view available data
ticker_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-02,58.525562,58.982404,58.048857,58.634808,4459200,0.0,0
2020-01-03,57.105375,57.721119,56.519424,57.661533,9078100,0.0,0
2020-01-06,56.598875,57.363587,56.271138,57.264275,5504300,0.0,0
2020-01-07,57.512559,57.671461,57.075580,57.214619,5563000,0.0,0
2020-01-08,57.353657,58.992334,57.333794,58.446106,8519000,0.0,0


In [14]:
# Calculate log returns
returns_df = ticker_data.assign(log_returns = lambda x: np.log(x.Close / x.Open))

returns_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,log_returns
Date,,,,,,,,
2020-01-02,58.525562,58.982404,58.048857,58.634808,4459200,0.0,0,0.001865
2020-01-03,57.105375,57.721119,56.519424,57.661533,9078100,0.0,0,0.009692
2020-01-06,56.598875,57.363587,56.271138,57.264275,5504300,0.0,0,0.011688
2020-01-07,57.512559,57.671461,57.075580,57.214619,5563000,0.0,0,-0.005194
2020-01-08,57.353657,58.992334,57.333794,58.446106,8519000,0.0,0,0.018868
...,...,...,...,...,...,...,...,...
2020-09-24,29.120001,29.510000,27.920000,29.010000,22257800,0.0,0,-0.003785
2020-09-25,28.820000,29.790001,28.780001,29.780001,21831500,0.0,0,0.032767
2020-09-28,30.600000,31.959999,30.139999,31.340000,22098400,0.0,0,0.023895


## Plot

In [18]:
fig = px.line(returns_df, y = "log_returns", title = "Title")
fig.show()